#Обучение классификатора на реальном датасете.

##Пояснение:
Требуется обучить классификатор определяющий тип велосипеда на небольшом датасете фотографий велосипедов из открытых источников. Допускается использовать пред-обученные модели из torchvision.models
По результатам работы составьте отчет.

##Задачи:

* Загрузите Датасет по ссылке: http://fmb.images.gan4x4.ru/hse/bt_dataset3.zip Используйте встроенные классы из пакета torchvision.datasets либо создайте свой являющийся наследником базового класса Dataset из этого пакета.
* Проведите аугментацию данных как минимум 3-мя различными способами. Хотя бы одни из них должен быть реализован самостоятельно.
* Используйте технологию Transfer learning: 
** измените выходной слой выбранной вами предобученной модели.
** Заморозьте часть весов.
* Обучите модель с использование GPU. Учитывайте особенности данных при выборе и настройке Loss-функции
*Оцените результаты.
* Напишите отчет.



*Сохраняйте веса модели на GoogleDrive

In [ ]:
! wget http://fmb.images.gan4x4.ru/hse/bt_dataset3.zip

In [ ]:
! unzip bt_dataset3.zip

In [ ]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import torch.utils.data as data

cudnn.benchmark = True
plt.ion()   # interactive mode

In [ ]:
# https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html#training-the-model

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'bike/bike_type'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=2) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed}')
    print(f'Best val accuracy: {best_acc}')
    
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, len(class_names))
model_ft = model_ft.to(device)
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=25)

Epoch 0/24
train Loss: 1.2980 Acc: 0.5035
val Loss: 0.8961 Acc: 0.7050

Epoch 1/24
train Loss: 1.1693 Acc: 0.5528
val Loss: 0.6249 Acc: 0.7983

Epoch 2/24
train Loss: 1.0909 Acc: 0.5969
val Loss: 0.5865 Acc: 0.8026

Epoch 3/24
train Loss: 0.9856 Acc: 0.6402
val Loss: 0.6997 Acc: 0.7549

Epoch 4/24
train Loss: 0.9332 Acc: 0.6480
val Loss: 0.6782 Acc: 0.8243

Epoch 5/24
train Loss: 0.9078 Acc: 0.6624
val Loss: 0.7683 Acc: 0.7592

Epoch 6/24
train Loss: 0.9020 Acc: 0.6699
val Loss: 0.6512 Acc: 0.7983

Epoch 7/24
train Loss: 0.6603 Acc: 0.7498
val Loss: 0.5985 Acc: 0.8265

Epoch 8/24
train Loss: 0.6038 Acc: 0.7725
val Loss: 0.5558 Acc: 0.8286

Epoch 9/24
train Loss: 0.6037 Acc: 0.7659
val Loss: 0.5729 Acc: 0.8460

Epoch 10/24
train Loss: 0.5501 Acc: 0.7921
val Loss: 0.6004 Acc: 0.8482

Epoch 11/24
train Loss: 0.5888 Acc: 0.7860
val Loss: 0.5543 Acc: 0.8373

Epoch 12/24
train Loss: 0.5457 Acc: 0.8000
val Loss: 0.5446 Acc: 0.8525

Epoch 13/24
train Loss: 0.5510 Acc: 0.7961
val Loss: 0.5695 A

In [ ]:
model_conv = torchvision.models.resnet18(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, len(class_names))
model_conv = model_conv.to(device)
criterion = nn.CrossEntropyLoss()
optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

In [ ]:
model_conv = train_model(model_conv, criterion, optimizer_conv, exp_lr_scheduler, num_epochs=25)

Epoch 0/24
train Loss: 1.3351 Acc: 0.4607
val Loss: 1.0383 Acc: 0.6074

Epoch 1/24
train Loss: 1.2918 Acc: 0.5031
val Loss: 1.2640 Acc: 0.5163

Epoch 2/24
train Loss: 1.1906 Acc: 0.5166
val Loss: 1.0853 Acc: 0.5922

Epoch 3/24
train Loss: 1.2130 Acc: 0.5127
val Loss: 1.0739 Acc: 0.5879

Epoch 4/24
train Loss: 1.1641 Acc: 0.5201
val Loss: 1.0383 Acc: 0.5792

Epoch 5/24
train Loss: 1.2076 Acc: 0.5223
val Loss: 0.8947 Acc: 0.6681

Epoch 6/24
train Loss: 1.1489 Acc: 0.5511
val Loss: 0.9913 Acc: 0.6356

Epoch 7/24
train Loss: 1.0123 Acc: 0.5825
val Loss: 0.9496 Acc: 0.6399

Epoch 8/24
train Loss: 1.0047 Acc: 0.5917
val Loss: 0.8797 Acc: 0.6377

Epoch 9/24
train Loss: 0.9784 Acc: 0.6013
val Loss: 0.9171 Acc: 0.6356

Epoch 10/24
train Loss: 0.9731 Acc: 0.6109
val Loss: 0.9147 Acc: 0.6334

Epoch 11/24
train Loss: 0.9808 Acc: 0.5965
val Loss: 0.9468 Acc: 0.6247

Epoch 12/24
train Loss: 1.0065 Acc: 0.5751
val Loss: 0.8913 Acc: 0.6508

Epoch 13/24
train Loss: 0.9796 Acc: 0.5869
val Loss: 0.9403 A

In [ ]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated
    
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title(f'predicted: {class_names[preds[j]]}')
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

visualize_model(model_conv)
plt.ioff()
plt.show()

#Отчет
Первая модель - Best val accuracy: 0.859002169197397

Вторая модель - Best val accuracy: 0.6811279826464208

Вывод: при переносе весов с помощью Transfer learning - accuracy сильно падате, но тем не менее 0.68 не плохой результат